<a href="https://colab.research.google.com/github/EACMichiels/AppForToolchain/blob/master/Granite_Docling_Document_Retrieval_RAG_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an AI-Powered Document Retrieval System with Docling and Granite

*Using IBM Granite Models*

## Recipe Overview

Welcome to this Granite recipe, in this recipe, you'll learn to harness the power of advanced tools to build AI-powered document retrieval systems. It will guide you through:

- **Document Processing:** Learn to handle documents from various sources, parse and transform them into usable formats, and store them in vector databases using Docling.
- **Retrieval-Augmented Generation (RAG):** Understand how to connect large language models (LLMs) like Granite with external knowledge bases to enhance query responses and generate valuable insights.
- **LangChain for Workflow Integration:** Discover how to use LangChain to streamline and orchestrate document processing and retrieval workflows, enabling seamless interaction between different components of the system.

This recipe leverages three cutting-edge technologies:

1. **[Docling](https://github.com/DS4SD/docling):** <span style="color: purple">An open-source toolkit for parsing and converting documents.</span>
2. **[Granite](https://www.ibm.com/granite/docs/models/granite/):** <span style="color: purple">A state-of-the-art LLM available via an [API](https://www.ibm.com/topics/api) through Replicate, providing robust natural language capabilities.</span>
3. **[LangChain](https://github.com/langchain-ai/langchain):** <span style="color: purple">A powerful framework for building applications powered by language models, designed to simplify complex workflows and integrate external tools seamlessly.</span>

<span style="color: purple; font-weight: bold; font-size: 20px">
Replicate is a Cloud platform that lets developers run and fine-tune Open-Source machine learning models via an API, without needing to set up any infrastructure.
</span>
<br>
<br>
By the end of this recipe, you will:
- Gain proficiency in document processing and chunking.
- Integrate vector databases to enhance retrieval capabilities.
- Utilize RAG to perform efficient and accurate data retrieval for real-world applications.

This recipe is designed for AI developers, researchers, and enthusiasts looking to enhance their knowledge of document management and advanced NLP techniques.



## Prerequisites

- Familiarity with Python programming.
- Basic understanding of large language models and natural language processing concepts.


## Step 1: Setting up the environment

Ensure you are running python 3.10, 3.11, or 3.12 in a freshly-created virtual environment.

In [ ]:
import sys
assert sys.version_info >= (3, 10) and sys.version_info < (3, 13), "Use Python 3.10, 3.11, or 3.12 to run this notebook."

## Step 2: Install dependencies

In [ ]:
! pip install "git+https://github.com/ibm-granite-community/utils.git" \
    transformers \
    langchain_community \
    langchain_huggingface \
    langchain_milvus \
    docling \
    replicate

  Cloning https://github.com/ibm-granite-community/utils.git to /tmp/pip-req-build-jrgc8jhw
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils.git /tmp/pip-req-build-jrgc8jhw
  Resolved https://github.com/ibm-granite-community/utils.git to commit 59db6fce5a2f50027ee7834b7b3127ed101fa221
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.1/166.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.7/141.7 kB 14.8 MB/s eta 0:00:00
   ━━━━

## Step 3: Selecting System Components

### Choose your Embeddings Model

Specify the model to use for generating embedding vectors from text. Here we will be using one of the new [Granite Embeddings models](https://huggingface.co/collections/ibm-granite/granite-embedding-models-6750b30c802c1926a35550bb)

To use a model from another provider, replace this code cell with one from [this Embeddings Model recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Embeddings_Models.ipynb).

<span style="color: purple; font-weight: bold; font-size: 20px">
A Tokenizer breaks down a text in smaller units, "tokens". This is often done in ML and NLP. An Autotokenizer chooses automatically the right Tokenizer for the Model at hand
.</span>

<span style="color: purple; font-weight: bold; font-size: 20px">
An embeddings model is a type of AI model that converts data (like text, images, or code) into numerical vectors in a high-dimensional space, such that similar inputs have similar vector representations.
</span>

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer

embeddings_model_path = "ibm-granite/granite-embedding-30m-english"
embeddings_model = HuggingFaceEmbeddings(
    model_name=embeddings_model_path,
)
embeddings_tokenizer = AutoTokenizer.from_pretrained(embeddings_model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/467k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### Use the Granite model

Select a Granite model from the [`ibm-granite`](https://replicate.com/ibm-granite) org on Replicate. Here we use the Replicate Langchain client to connect to the model.

To get set up with Replicate, see [Getting Started with Replicate](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started/Getting_Started_with_Replicate.ipynb).

To connect to a model on a provider other than Replicate, substitute this code cell with one from the [LLM component recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_LLMs.ipynb).

In [ ]:
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var

model_path = "ibm-granite/granite-3.2-8b-instruct"
model = Replicate(
    model=model_path,
    replicate_api_token=get_env_var("REPLICATE_API_TOKEN"),
    model_kwargs={
        "max_tokens": 1000, # Set the maximum number of tokens to generate as output.
        "min_tokens": 100, # Set the minimum number of tokens to generate as output.
    },
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

REPLICATE_API_TOKEN loaded from Google Colab secret.


tokenizer_config.json:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

Now that we have the model downloaded, let's try asking it a question

<span style="color: purple; font-weight: bold; font-size: 20px">
UFC 310 or Ultimate Fighting Championship 310: Pantoja vs. Asakura was a mixed martial arts event that took place on December 7, 2024, at the T-Mobile Arena in Paradise, Nevada, part of the Las Vegas Valley.
</span>

In [ ]:
from langchain.prompts import PromptTemplate

query = "Who won in the Pantoja vs Asakura fight at UFC 310?"

# Create a Granite prompt for question-answering
prompt = tokenizer.apply_chat_template(
    conversation=[{
        "role": "user",
        "content": "{input}",
    }],
    add_generation_prompt=True,
    tokenize=False,
)
prompt_template = PromptTemplate.from_template(template=prompt)

chain = prompt_template | model

output = chain.invoke({"input": query})

print(output)

The fight between Marlon Moraes (Pantoja) and John Lineker (Asakura) took place at UFC 310, not a fight between Pantoja and Asakura. 

Marlon Moraes won the fight against John Lineker via unanimous decision.


<span style="color: purple; font-weight: bold; font-size: 20px">
This does not seem to be the right fighters. The model doesn't seem to know the exact answer. Let's see if it has some specific UFC rules info.
</span

In [ ]:
query1 = "How much weight allowance is allowed in non championship fights in the UFC?"

output = chain.invoke({"input": query1})

print(output)

In non-championship fights within the Ultimate Fighting Championship (UFC), fighters are generally allowed a weight allowance of 1 pound (0.45 kg) for each day leading up to the weigh-in, which typically occurs 24 hours before the fight. This rule is known as the "one-pound allowance per day" policy. 

However, there are exceptions and special considerations:

1. If a fighter misses weight, they are usually ineligible for certain bonuses (like "Fight of the Night" or "Performance of the Night"). 

2. In some cases, the UFC may grant a fighter more leeway if they are moving up or down a weight class for a specific fight. 

3. For international fights, local commission rules might apply and could potentially have different weight allowance regulations.

It's always best to check the most recent UFC rules and regulations or consult with the UFC directly for the most accurate and up-to-date information, as these rules can change over time.


<span style="color: purple; font-weight: bold; font-size: 20px">
Based on the official UFC rules, this is also incorrect. Let's try getting some documents that contains this information for the model.
</span>

### Choose your Vector Database

Specify the database to use for storing and retrieving embedding vectors.

To connect to a vector database other than Milvus, replace this code cell with one from [this Vector Store recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Vector_Stores.ipynb).

In [ ]:
from langchain_milvus import Milvus
import tempfile

db_file = tempfile.NamedTemporaryFile(prefix="milvus_", suffix=".db", delete=False).name
print(f"The vector database will be saved to {db_file}")

vector_db = Milvus(
    embedding_function=embeddings_model,
    connection_args={"uri": db_file},
    auto_id=True,
    enable_dynamic_field=True,
    index_params={"index_type": "AUTOINDEX"},
)

The vector database will be saved to /tmp/milvus_nkw_it1n.db


## Step 4: Building the Vector Database

In this example, from a set of source documents, we use [Docling](https://github.com/DS4SD/docling) to convert the documents into text and then split the text into chunks, derive embedding vectors using the embedding model, and load it into the vector database.
<br>
<br>
<span style="color: purple; font-weight: bold; font-size: 20px">
Creating this vector database will allow us to easily search across our documents, enabling us to use RAG.
</span>

### Use Docling to download the documents, convert to text, and split into chunks

Here we have found a website that gives us information on UFC 310, as well as a PDF of the official UFC rules. Below, we will see that Docling can both convert and chunk the two documents.

In [ ]:
# Docling imports
from docling.document_converter import DocumentConverter
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker
from docling_core.types.doc.labels import DocItemLabel
from langchain_core.documents import Document

# Here are our documents, feel free to add more documents in formats that Docling supports
sources = [
    "https://www.ufc.com/news/main-card-results-highlights-winner-interviews-ufc-310-pantoja-vs-asakura",
    "https://media.ufc.tv/discover-ufc/Unified_Rules_MMA.pdf",
]

converter = DocumentConverter()

# Convert and chunk out documents
i = 0
texts: list[Document] = [
    Document(page_content=chunk.text, metadata={"doc_id": (i:=i+1), "source": source})
    for source in sources
    for chunk in HybridChunker(tokenizer=embeddings_tokenizer).chunk(converter.convert(source=source).document)
    if any(filter(lambda c: c.label in [DocItemLabel.TEXT, DocItemLabel.PARAGRAPH], iter(chunk.meta.doc_items)))
]

print(f"{len(texts)} document chunks created")

Token indices sequence length is longer than the specified maximum sequence length for this model (556 > 512). Running this sequence through the model will result in indexing errors


17 document chunks created


In [ ]:
# Print all created documents
for document in texts:
    print(f"Document ID: {document.metadata['doc_id']}")
    print(f"Source: {document.metadata['source']}")
    print(f"Content:\n{document.page_content}")
    print("=" * 80)  # Separator for clarity

Document ID: 1
Source: https://www.ufc.com/news/main-card-results-highlights-winner-interviews-ufc-310-pantoja-vs-asakura
Content:
Events
Upcoming
Past
Tickets
VIP Experiences
Group Sales
UFC Travel Deals
UFC Fight Pass Invitational
Road to UFC
Dana White's Contender Series
Rankings
Athletes
All Athletes
Hall of Fame
Record Book
News
UFC
Connect
Newsletter
UFC Fight Club
UFC Apex
Find a Gym
EA Sports UFC 5
Betting Odds
Watch
How to Watch
Find a Bar
UFC Fight Pass
UFC Video Archive
SHOP
UFC STORE
UFC COLLECTIBLES
VENUM FIGHT WEEK
UFC STRIKE
MORE
Thorne Performance Solutions
Results
Document ID: 2
Source: https://www.ufc.com/news/main-card-results-highlights-winner-interviews-ufc-310-pantoja-vs-asakura
Content:
See The Fight Results, Watch Post-Fight Interviews With The Main Card Winners And More From UFC 310: Pantoja vs Asakura, Live From T-Mobile Arena In Las Vegas, Nevada
By E. Spencer Kyte, On X @spencerkyte
 • Dec. 8, 2024
The UFC 310 preliminary card slate was outstanding, featurin

<span style="color: purple; font-weight: bold; font-size: 20px">
You got 16 documents!
</span>

### Populate the vector database

NOTE: Population of the vector database may take over a minute depending on your embedding model and service.

In [ ]:
ids = vector_db.add_documents(texts)
print(f"{len(ids)} documents added to the vector database")

17 documents added to the vector database


## Step 5: RAG with Granite

Now that we have succesfully converted our documents and vectorized them, we can set up out RAG pipeline.

### Retrieve relevant chunks



Here we will test the as_retriever method to search through our newly created vector database for chunks that are relevant to our original query



In [ ]:
retriever = vector_db.as_retriever()

docs = retriever.invoke(query)
print(docs)

[Document(metadata={'doc_id': 12, 'source': 'https://www.ufc.com/news/main-card-results-highlights-winner-interviews-ufc-310-pantoja-vs-asakura', 'pk': 457619686425100299}, page_content='Alexandre Pantoja Post-Fight Interview | UFC 310\nThe champion came out aggressively to start the fight, seeking to bring the action to Asakura. He stung the newcomer in the first and was happy to exchange through much of the first two rounds before jumping on Asakura’s back and sinking him to the canvas. From there, Pantoja attacked the choke and secured the finish.\nThat’s now three consecutive successful title defenses for the Brazilian, and arguably the most impressive effort to date inside the Octagon. The American Top Team representative remains a cut above the competition in the 125-pound weight class, and heads into next year riding a seven-fight winning streak.\nPrelim Results | Official Scorecards\nUFC 310: Pantoja vs Asakura took place live from T-Mobile Arena in Las Vegas, Nevada on Decembe

Looks like it pulled some chunks that would have the information we are looking for. Let's go ahead and contruct our RAG pipeline.

### Create the prompt for Granite

Next, we construct the prompt pipeline. This creates the prompt which holds the retrieved chunks from out previous search and feeds this to the model as context for answering our question.

In [ ]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create a Granite prompt for question-answering with the retrieved context
prompt = tokenizer.apply_chat_template(
    conversation=[{
        "role": "user",
        "content": "{input}",
    }],
    documents=[{
        "title": "placeholder",
        "text": "{context}",
    }],
    add_generation_prompt=True,
    tokenize=False,
)
prompt_template = PromptTemplate.from_template(template=prompt)

# Create a Granite document prompt template to wrap each retrieved document
document_prompt_template = PromptTemplate.from_template(template="""\
Document {doc_id}
{page_content}""")
document_separator="\n\n"

# Assemble the retrieval-augmented generation chain
combine_docs_chain = create_stuff_documents_chain(
    llm=model,
    prompt=prompt_template,
    document_prompt=document_prompt_template,
    document_separator=document_separator,
)
rag_chain = create_retrieval_chain(
    retriever=vector_db.as_retriever(),
    combine_docs_chain=combine_docs_chain,
)

### Generate a retrieval-augmented response to a question

The pipeline uses the query to locate documents from the vector database and use them as context for the query.

In [ ]:
output = rag_chain.invoke({"input": query})

print(output['answer'])

Alexandre Pantoja won the fight against Kai Asakura at UFC 310.


Awesome! It looks like the model figured out our first question. Let's see if it figure out the rule we were looking for.

In [ ]:
output = rag_chain.invoke({"input": query1})

print(output['answer'])

In non-championship fights in the UFC, there is a 1-pound weight allowance.


Awesome! We can now see that we have created a pipeline that can successfully leverage knowledge from multiple document types for generation.

## Next Steps

- Explore advanced RAG workflows for other industries
- Experiment with other document types and larger datasets.
- Optimize prompt engineering for better Granite responses.

Thank you for using this recipe!